In [7]:
import json
import random
from numpy import linspace
import itertools

with open('ciencias_horarios/materias_limpio.json') as archivo:
    materias_limpio = json.load(archivo)

Hacemos la clase de materia, donde cada objeto representará una materia y contendrá información acerca de sus grupos, horarios y profesores

In [8]:
class materia():
    '''
    Clase que contendrá la información de una materia
    '''
    def __init__(self, clave,materia):
        '''
        Función de inicialización que creará el diccionario con los 
        grupos de la materia, y un diccionario cuyas llaves serán los días
        de la semana y a su vez cada una contendrá un diccionario con las
        horas y los grupos que imparten la materia en cada hora.
        -------------------------------------------------------------------
        :param clave str: Cadena de cuatro dígitos que contiene la clave
                         de la materia
        :param materia str: Cadena con el nombre de la materia
        '''
        self.grupos = {}
        self.clave = clave
        self.nombre = materia
        self.horarios = {
            'lu':{},
            'ma':{},
            'mi':{},
            'ju':{},
            'vi':{},
            'sa':{}
        }
        
    def agregar_grupo(self,grupo,profesor,horario):
        '''
        Función de agregació  de grupos a la clase. Agregará
        el grupo al diccionario `grupos` así como actualización
        del diccionario de horarios.
        -----------------------------------------
        :param grupo str: Cadena de 4 dígitos con el número de grupo.
        :param profesor str: Nombre del docente del grupo.
        :param horario dict: Diccionario que incluye el horario por día
        '''
        for dia in horario.keys():
            #print(horario[dia])
            if horario[dia] !=[]:
                hora_dia = tuple(horario[dia][0])

                if hora_dia not in list(self.horarios[dia].keys()):

                    self.horarios[dia][hora_dia] = []
                self.horarios[dia][hora_dia].append(grupo)

        self.grupos[grupo] = [profesor,horario]

Y guardamos cada materia como un objeto de la clase:

In [9]:
dict_materias = {}
claves = []
grupos_claves = {}
for materia_limpio in materias_limpio:
    clave = materia_limpio['clave']
    if clave not in claves:
        claves.append(clave)
        asignatura = materia_limpio['materia']
        mat = materia(clave,asignatura)
        dict_materias[clave] = mat
    
    grupo = materia_limpio['grupo']
    profesor = materia_limpio['profesor']
    horario = materia_limpio['horario']
    grupos_claves[grupo] = clave
    dict_materias[clave].agregar_grupo(grupo,profesor,horario)

Algunas funciones para pasar de claves a nombres y grupos, así como otras funciones útiles

In [10]:
def str_to_clave(clave):
    '''
    Función que convierte un entero a una clave 
    en cadena de texto 4 dígitos
    '''
    clave_str = str(clave)
    clave_str = '0'*(4-len(clave_str))+clave_str
    return clave_str
    
def clave_to_nombre(clave):
    '''
    Dada una clave , regresa el nombre de la asignatura
    '''
    if isinstance(clave, int):
        clave_str = str_to_clave(clave)
        return dict_materias[clave_str].nombre 
        
    return dict_materias[clave].nombre


def get_horario(clave,hora_inicio,dia):
    '''
    Dada una clave, regresa todos los grupos que comiencen a 
    la hora dada.
    
    :param clave str: Cadena de 4 dígitos que indica la materia
    :param hora_inicio float: Hora a la que inicia la clase
    :para dia str: Cadena de dos letras que indica el día del horario.
    
    :returns False: En caso de que no haya algún grupo en ese horario
    :returns (horario_inicio,horario_fin), grupos:
                    Regresa una tupla con inicio y fin de la materio junto
                    a sus grupos.
    '''
    materia = dict_materias[clave]
    for horario in materia.horarios[dia].keys():
        if hora_inicio == horario[0]:
            return horario, materia.horarios[dia][horario]
    return False

def grupo_to_clave(grupo):
    '''
    Regresa a qué materia pertenece un grupo específico
    -----------------------------------------------------
    :param grupo str: Cadena de 4 dígitos de un grupo
    
    :returns str: Cadena de 4 dígitos que representa la materia 
    '''
    return grupos_claves[grupo]


La siguiente función nos permite obtener toda la información de una materia con solo el código de grupo

In [17]:

def grupo_info(grupo):
    '''
    Da la información de un grupo
    --------------------------------------
    :param grupo str: Cadena de 4 dígitos de un grupo
    
    :returns: Profesor, Horario de la materia
    '''
    clave = grupo_to_clave(grupo)
    grupo = dict_materias[clave].grupos[grupo]
    profesor , horario = grupo
    horario = {dia:tuple(horario[dia][0]) for dia in horario.keys() if horario[dia] !=[]}
    return clave, profesor, horario

grupo_info('4117')

('0005',
 'alejandro alvarado garcia',
 {'lu': (10.0, 11.0),
  'ma': (10.0, 11.0),
  'mi': (10.0, 11.0),
  'ju': (10.0, 11.0),
  'vi': (10.0, 11.0)})

## Recomendador de horarios

La siguiente función crea candidatos entre la hora de inicio y fin de un día:

In [12]:
def generar_candidatos(claves,hora_inicio,hora_fin,dia):
    '''
    Para las claves e información sobre hora y día, regresa un diccionario con
    los grupos disponibles para esas materias en esas restricciones, es decir
    un diccionario cuyas llaves son intervalos de horas y sus valores los grupos
    que tienen clase a esa hora en ese día, eligiendo intervalos de horas entre
    la hora de inicio y fin establecidas.
    -------------------------------------------------
    :param claves [str]: Lista con las claves de materias a recuperar
    :hora_inicio float: Hora a la que inicia el horario debe ser entero o
                        un entero y medio (e.g 9.0 y 9.50 son válidos, mientras que
                        6:35 no es una hora válida) el 0.5 representa la media
                        hora.
    :hora_fin float: Hora máxima a la que termina el horario
    
    :returns horario_dia dict: Diccionario que tiene como llaves las horas que tienen candidatos,
                                con los candidatos a dicha hora
    :returns horas_disponibles list: Lista con los intervalos de horas que se consideran para los 
                                candidatos, es decir cada grupo candidato imparte su clase en alguna
                                hora de esta lista.
    :returns grupos_por_clave dict: Diccionario que tiene como llaves las claves de las materias y
                                como valores, los grupos candidatos que se encontraron para cada
                                materia
    '''
    if hora_inicio%0.5 != 0 or hora_fin%0.5 != 0:
        raise Exception('Ese no es un horario válido, tiene que representar una hora o \n fracción de media hora')
        
    
        
    horario_dia = {}
    horas_disponibles = set()
    grupos_por_clave = {}
    #Un intervalo de las horas representadas en fracciones de media hora
    horas_eleccion =  linspace(hora_inicio,hora_fin,int(2*(hora_fin-hora_inicio)+1))
    
    #for dia in dias.keys():
    for clave in claves:
        grupos_por_clave[clave] = []
        for hora in horas_eleccion:
            horario = get_horario(clave,hora,dia)

            if horario is False: #si no hay grupos disponibles a esa hora
                continue
            if horario[0][1]>hora_fin: #Si el horario se pasa de la hora establecida
                continue

            hora_de_clase = horario[0]
            grupos = horario[1]
            if hora_de_clase not in horario_dia.keys():
                horario_dia[hora_de_clase] = []
                
            horas_disponibles.add(hora_de_clase)
            horario_dia[hora_de_clase] += grupos
            grupos_por_clave[clave] += grupos
    
    return horario_dia,list(horas_disponibles), grupos_por_clave

candidatos = generar_candidatos(['0002','0003','0083','0163','0715'],8.0,12,'lu')
candidatos

({(10.0, 11.0): ['4212', '4213'], (9.0, 10.0): ['4214', '4214']},
 [(9.0, 10.0), (10.0, 11.0)],
 {'0002': ['4212', '4213'],
  '0003': ['4214', '4214'],
  '0083': [],
  '0163': [],
  '0715': []})

Y a continuación la función para recomendar candidatos no solo para un día, sino para toda la semana:

In [20]:
def generar_candidatos_semana(claves, hora_inicio, hora_fin):
    '''
    Función que da información acerca de los grupos que imparten clase
    entre dos horas establecidas.
    ---------------------------------------------------------------
    :param claves [str]: Lista con las claves de las materias a acomodar
    :param hora_inicio float: Hora a la que inicia el horario debe ser entero o
                        un entero y medio (e.g 9.0 y 9.50 son válidos, mientras que
                        6:35 no es una hora válida) el 0.5 representa la media
                        hora.
    :param hora_fin float: Hora máxima a la que termina el horario
    
    
    :returns horario_por_dia dict: Diccionario que tiene como llaves los días que tienen candidatos,
                                con los candidatos por intervalo de hora
    :returns horas_disponibles dict: Diccionario con los días e intervalos de horas que se consideran para los 
                                candidatos, es decir cada grupo candidato imparte su clase en alguna
                                hora de este diccionario.
    :returns grupos_por_clave dict: Diccionario que tiene como llaves las claves de las materias y
                                como valores, los grupos candidatos que se encontraron para cada
                                materia
    '''
    dias = ['lu','ma','mi','ju','vi','sa']
    
    horario_por_dia = {}
    horas_disponibles_por_dia = {}
    grupos_por_clave = {}
    for dia in dias:
        candidatos_dia = generar_candidatos(claves,hora_inicio,hora_fin,dia)
        horario_por_dia[dia] = candidatos_dia[0]
        horas_disponibles_por_dia[dia] = candidatos_dia[1]
        for intervalo in candidatos_dia[2].keys():
            if intervalo not in grupos_por_clave.keys():
                grupos_por_clave[intervalo] = []
            grupos_por_clave[intervalo]+= candidatos_dia[2][intervalo]
    
    grupos_por_clave = {intervalo:list(set(grupos_por_clave[intervalo]))
                       for intervalo in grupos_por_clave.keys()}
    return horario_por_dia, horas_disponibles_por_dia, grupos_por_clave

candidatos = generar_candidatos_semana(['0002','0001','0715'],8.0,11)
candidatos

({'lu': {(10.0, 11.0): ['4212', '4213'], (9.0, 10.0): ['4175', '4176']},
  'ma': {(10.0, 11.0): ['4212', '4213'], (9.0, 10.0): ['4175', '4176']},
  'mi': {(10.0, 11.0): ['4212', '4213'], (9.0, 10.0): ['4175', '4176']},
  'ju': {(10.0, 11.0): ['4212', '4213'], (9.0, 10.0): ['4175', '4176']},
  'vi': {(10.0, 11.0): ['4212', '4213'], (9.0, 10.0): ['4175', '4176']},
  'sa': {}},
 {'lu': [(9.0, 10.0), (10.0, 11.0)],
  'ma': [(9.0, 10.0), (10.0, 11.0)],
  'mi': [(9.0, 10.0), (10.0, 11.0)],
  'ju': [(9.0, 10.0), (10.0, 11.0)],
  'vi': [(9.0, 10.0), (10.0, 11.0)],
  'sa': []},
 {'0002': ['4212', '4213'], '0001': ['4175', '4176'], '0715': []})

A continuación se definen funciones para acomodar horarios en las que se verifica si un intervalo de tiempo interfiere con un horario establecido para un día, después lo mismo pero intervalos de tiempo en una semana, y finalmente una función que compara los horarios a las que se imparten materias una lista de grupos y verifica si alguna se intersecta con algún otro grupo

In [22]:
def se_intersecta(intervalo,horario_dia):
    '''
    Dado un intervalo de tiempo y un horario, checa si alguna
    clase del horario se intersecta con el intervalo.
    
    :param intervalo tuple: Intervalo de tiempo a verificar
    :param hoarario dict: Diccionario que tiene como llaves días de la semana
                        y horas a las que se imparten materias desglosadas por
                        hora. Ejem:
            {'lu': {(10.0, 11.0): ['4212', '4213'], (9.0, 10.0): ['4175', '4176']},
              'ma': {(10.0, 11.0): ['4212', '4213'], (9.0, 10.0): ['4175', '4176']},
              'mi': {(10.0, 11.0): ['4212', '4213'], (9.0, 10.0): ['4175', '4176']},
              'ju': {(10.0, 11.0): ['4212', '4213'], (9.0, 10.0): ['4175', '4176']},
              'vi': {(10.0, 11.0): ['4212', '4213'], (9.0, 10.0): ['4175', '4176']},
              'sa': {}}
    
    :returns Bool: Verdadero si se intersectan, falso en otro caso.
    '''
    horario_lista = sorted(list(horario_dia.keys()),key= lambda x:x[1])
    inicio = intervalo[0]
    fin = intervalo[1]
    for hora in horario_lista:
        hora_inicio = hora[0]
        hora_fin = hora[1]
        if hora_inicio <= inicio < hora_fin or hora_inicio < fin <= hora_fin:
            return True
    return False

def se_intersecta_semana(intervalos,horario):
    '''
    Dado un horario de materia por día y un horario, checa si alguna
    clase del horario se intersecta con la materia.
    -----------------------------------------------------------------
    :param intervalo tuple: Horario de un grupo a verificar.
                               Ejem: {'lu': (10.0, 11.0),
                                      'ma': (10.0, 11.0),
                                      'mi': (10.0, 11.0),
                                      'ju': (10.0, 11.0),
                                      'vi': (10.0, 11.0)}
    :param hoarario dict: Diccionario que tiene como llaves días de la semana
                        y horas a las que se imparten materias desglosadas por
                        hora. Ejem:
            {'lu': {(10.0, 11.0): ['4212', '4213'], (9.0, 10.0): ['4175', '4176']},
              'ma': {(10.0, 11.0): ['4212', '4213'], (9.0, 10.0): ['4175', '4176']},
              'mi': {(10.0, 11.0): ['4212', '4213'], (9.0, 10.0): ['4175', '4176']},
              'ju': {(10.0, 11.0): ['4212', '4213'], (9.0, 10.0): ['4175', '4176']},
              'vi': {(10.0, 11.0): ['4212', '4213'], (9.0, 10.0): ['4175', '4176']},
              'sa': {}}
    
    :returns Bool: Verdadero si se intersectan, falso en otro caso.
    '''
    dias = ['lu','ma','mi','ju','vi','sa']
    for dia in horario.keys():
        if dia in intervalos.keys():
            horario_dia = horario[dia]
            if se_intersecta(intervalos[dia],horario_dia):
                return True
    return False

prueba_dias = candidatos[0]
horario_prueba = grupo_info('4174')[2]
se_intersecta_semana(horario_prueba,prueba_dias)

False

In [23]:
def conjunto_se_intersecta(grupos):
    '''
    Dado un conjunto de grupos, comprueba si los horarios
    de los grupos se intersectan
    -------------------------------------------------
    :param grupos [str]: Lista de grupos
    
    :return Bool: Indicador de si los conjuntos se intersectan o no.
    :returns Horario: Horario desglosado por día y hora
    '''
    dias = ['lu','ma','mi','ju','vi','sa']
    primer_horario = grupo_info(grupos[0])[2]
    horario = {dia:{} for dia in dias}
    
    for dia, hora in primer_horario.items():
        horario[dia][hora] = grupos[0]
    
    for grupo in grupos[1:]:
        horario_materia = grupo_info(grupo)[2]
        if se_intersecta_semana(horario_materia,horario):
            return True, horario
        for dia, hora in horario_materia.items():
            horario[dia][hora] = grupo

    return False, horario
conjunto_se_intersecta(['4171','4172'])

(False,
 {'lu': {(15.0, 16.0): '4171', (19.0, 20.0): '4172'},
  'ma': {(15.0, 16.0): '4171', (19.0, 20.0): '4172'},
  'mi': {(15.0, 16.0): '4171', (19.0, 20.0): '4172'},
  'ju': {(15.0, 16.0): '4171', (19.0, 20.0): '4172'},
  'vi': {(15.0, 16.0): '4171', (19.0, 20.0): '4172'},
  'sa': {}})